In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import datetime
import pandas as pd
import random

from matplotlib import pyplot as plt

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Conv2DTranspose, Reshape, UpSampling2D, LeakyReLU, LayerNormalization, Add
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam


from tensorflow.keras import layers
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array


def plot_losses(history):
    plt.rcParams['figure.figsize'] = [20, 5]
    f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)

    ax1.set_title('Losses')
    ax1.set_xlabel('epoch')
    ax1.legend(loc="upper right")
    ax1.grid()
    ax1.plot(history['loss'], label='Training loss')
    ax1.plot(history['val_loss'], label='Validation loss')
    ax1.legend()

    ax2.set_title('Accuracy')
    ax2.set_xlabel('epoch')
    ax2.legend(loc="upper right")
    ax2.grid()
    ax2.plot(history['accuracy'], label='Training accuracy')
    ax2.plot(history['val_accuracy'], label='Validation accuracy')
    ax2.legend()

    plt.show()

def plot_resultados(model, carpeta, height=64, width=64, n=4):
    """
    Muestra comparaciones entre imágenes originales y reconstruidas por el autoencoder.
    """
    # Seleccionar imágenes aleatorias
    archivos = os.listdir(carpeta)
    archivos_img = random.sample(archivos, n)

    # Cargar y normalizar las imágenes
    imgs_originales = []
    for nombre in archivos_img:
        img = load_img(os.path.join(carpeta, nombre), target_size=(height, width))
        img_array = img_to_array(img) / 255.0  # normalizar a [0,1]
        imgs_originales.append(img_array)

    imgs_originales = np.array(imgs_originales)

    # Reconstruir con el modelo
    imgs_reconstruidas = model.predict(imgs_originales)

    # Mostrar resultados
    plt.figure(figsize=(12, 6))
    for i in range(n):
        # Imagen original
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(imgs_originales[i])
        ax.set_title("Original")
        ax.axis("off")

        # Imagen reconstruida
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(imgs_reconstruidas[i])
        ax.set_title("Reconstruida")
        ax.axis("off")

    plt.tight_layout()
    plt.show()


def mostrar_imagenes_generadas(modelo_generador, latent_dim=100, num_imagenes=4, image_size=(64, 64, 3)):
    """
    Genera y muestra imágenes lado a lado usando un modelo generador.

    Args:
        modelo_generador: modelo Keras que recibe un vector latente y devuelve una imagen.
        latent_dim: dimensión del vector latente.
        num_imagenes: número de imágenes a generar.
        image_size: tamaño esperado de las imágenes (solo usado si quieres validar forma).
    """
    # Generar vectores latentes aleatorios
    z = generateLatentDim(latent_dim, num_imagenes)

    # Generar imágenes con el modelo
    imagenes = modelo_generador.predict(z)

    # Mostrar imágenes en una fila
    fig, axes = plt.subplots(1, num_imagenes, figsize=(num_imagenes * 3, 3))
    for i, ax in enumerate(axes):
        ax.imshow((imagenes[i] + 1) / 2)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

def generar_imagenes(modelo_generador, latent_dim=100, num_imagenes=4):
    """
    Genera imágenes en escala de grises usando un modelo generador y OpenCV.

    Args:
        modelo_generador: modelo Keras que recibe un vector latente y devuelve una imagen.
        latent_dim: dimensión del vector latente.
        num_imagenes: número de imágenes a generar.

    Returns:
        Array de imágenes en escala de grises con forma (num_imagenes, alto, ancho).
    """
    # Generar vectores latentes aleatorios
    z = np.random.normal(0, 1, (num_imagenes, latent_dim))

    # Generar imágenes con el modelo
    imagenes_rgb = modelo_generador.predict(z)

    # Asegurar que las imágenes están en el rango [0, 255] y tipo uint8
    imagenes_rgb = (imagenes_rgb * 255).astype(np.uint8)

    # Convertir a escala de grises con OpenCV
    imagenes_grises = np.array([cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY) for imagen in imagenes_rgb])

    return imagenes_grises


# GAN

## Defining Hiperparameters

In [ ]:
import os

os.listdir("/kaggle/input")

In [ ]:
height, width = 64, 64 # Image size, to fit in the competition size
image_dir = '/kaggle/input'
folder = None

latent_dim_size = 100
batch_size = 128

In [ ]:
# Latent dim generator
def generateLatentDim(latent_dim_size=latent_dim_size, batch_size= batch_size):
    return np.random.randn(batch_size, latent_dim_size)

def build_discriminator(img_shape=(64, 64, 3)):
    img_input = Input(shape=img_shape, name="image_input")

    # 64x64 -> 32x32
    x = Conv2D(64, kernel_size=3, strides=2, padding='same')(img_input)
    x = LeakyReLU(0.2)(x)

    # 32x32 -> 16x16
    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(0.2)(x)

    # 16x16 -> 8x8
    x = Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.3)(x)

    # 8x8 -> 4x4
    x = Conv2D(512, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.3)(x)

    # Clasificación final
    x = Flatten()(x)
    out = Dense(1, activation='sigmoid')(x)

    return keras.Model(img_input, out, name="Discriminator")


def build_generator(latent_dim=100, channels=3):
    z = Input(shape=(latent_dim,), name="latent_vector")

    x = Dense(512 * 8 * 8)(z)
    x = Reshape((8, 8, 512))(x)
    x = LeakyReLU(0.2)(x)

    # 8x8 -> 16x16
    x = Conv2DTranspose(256, kernel_size=3, strides=2, padding='same')(x)
    x = LayerNormalization()(x)
    x = LeakyReLU(0.2)(x)

    # 16x16 -> 32x32
    x = Conv2DTranspose(128, kernel_size=3, strides=2, padding='same')(x)
    x = LayerNormalization()(x)
    x = LeakyReLU(0.2)(x)

    # 32x32 -> 64x64
    x = Conv2DTranspose(64, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(0.2)(x)

    # Salida 64x64x3 en rango [-1, 1]
    out = layers.Conv2D(channels, kernel_size=3, padding='same', activation='tanh')(x)

    return keras.Model(z, out, name="Generator")


In [ ]:
def residual_block(x, filters, upsample=False):
    shortcut = x

    if upsample:
        # Upsample input and shortcut
        x = Conv2DTranspose(filters, kernel_size=3, strides=2, padding='same')(x)
        shortcut = Conv2DTranspose(filters, kernel_size=1, strides=2, padding='same')(shortcut)
    else:
        x = Conv2D(filters, kernel_size=3, padding='same')(x)

    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(filters, kernel_size=3, padding='same')(x)

    # Match shortcut shape if needed
    if not upsample and shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=1, padding='same')(shortcut)

    x = Add()([x, shortcut])
    x = LeakyReLU(0.2)(x)
    return x

def build_megaGenerator(latent_dim=100, channels=3):
    z = Input(shape=(latent_dim,), name="latent_vector")

    x = Dense(1024 * 4 * 4)(z)
    x = Reshape((4, 4, 1024))(x)
    x = LeakyReLU(0.1)(x)

    # 4x4 → 8x8 (residual upsample)
    x = residual_block(x, 512, upsample=True)

    # 8x8 → 16x16 (normal upsample)
    x = Conv2DTranspose(256, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.1)(x)

    # 16x16 → 32x32 (residual upsample)
    x = residual_block(x, 128, upsample=True)

    # 32x32 → 64x64 (normal upsample)
    x = Conv2DTranspose(64, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.1)(x)

    # Output layer
    out = Conv2D(channels, kernel_size=3, padding='same', activation='tanh')(x)

    return keras.Model(z, out, name="Generator")

We define the autoencoder, encoder and decoder

In [ ]:
if folder == None:
    generator = build_megaGenerator()
    discriminator = build_discriminator()
else:
    generator = keras.models.load_model(folder + "Generator.keras")
    discriminator = keras.models.load_model(folder + "Discrimiator.keras")
generator.summary()

## ImageDataGenerator

To fit with the competition, we will reshape images to 64x64

In [ ]:
# Función personalizada para escalar entre -1 y 1
def scale_minus1_to_1(img):
    return img / 127.5 - 1.0

datagen = ImageDataGenerator(preprocessing_function=scale_minus1_to_1)

train_generator = datagen.flow_from_directory(
    image_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='input',
    subset='training',
    shuffle=True
)


## Train

In [ ]:
# Compilar discriminador con binary_crossentropy
opt_d = keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
opt_g = keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
discriminator.trainable = True
discriminator.compile(optimizer=opt_d, loss='binary_crossentropy', metrics=['accuracy'])

# Congelar discriminador para entrenar el generador
discriminator.trainable = False
gan_input = tf.keras.layers.Input(shape=(latent_dim_size,))
gan_output = discriminator(generator(gan_input))
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(optimizer=opt_g, loss='binary_crossentropy')

In [ ]:
epochs = 10000

# Bucle de entrenamiento
for epoch in range(epochs):
    # === Entrenar discriminador ===
    # 1. Obtener imágenes reales del generador
    real_images = next(train_generator)[0]

    # 2. Generar imágenes falsas
    z = generateLatentDim()
    fake_images = generator(z, training=False)

    # 3. Crear etiquetas: 1 para reales, 0 para falsas
    real_labels = np.ones((real_images.shape[0], 1))
    fake_labels = np.zeros((fake_images.shape[0], 1))

    # # 4. Combinar y mezclar
    # combined_images = np.concatenate([real_images, fake_images], axis=0)
    # combined_labels = np.concatenate([real_labels, fake_labels], axis=0)

    # indices = np.arange(combined_images.shape[0])
    # np.random.shuffle(indices)

    # shuffled_images = combined_images[indices]
    # shuffled_labels = combined_labels[indices]

    # 5. Entrenar discriminador con batch mezclado (solo una de cada 2)
    # discriminator.trainable = True
    # d_loss = discriminator.train_on_batch(shuffled_images, shuffled_labels)
    if epoch % 3 == 0:
        discriminator.trainable = True
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


    # === Entrenar generador ===
    z = generateLatentDim()
    trick_labels = np.ones((batch_size, 1))  # queremos que el discriminador crea que son reales
    discriminator.trainable = False
    g_loss = gan.train_on_batch(z, trick_labels)

    # Mostrar progreso
    if epoch % 200 == 0:
        print(f"Epoch {epoch} | D loss: {d_loss[0]:.4f} | D acc: {d_loss[1]:.4f} | G loss: {g_loss:.4f}")
        mostrar_imagenes_generadas(generator)


### Comprobar resultados

In [ ]:
mostrar_imagenes_generadas(generator)

In [ ]:
# Current timestamp
timestamp = datetime.datetime.now().strftime("%m_%d_%H:%M")
## Make sure everything saves correctly
os.makedirs("models", exist_ok=True)
os.makedirs("models/"+timestamp, exist_ok=True)
generator_path = f"models/{timestamp}/Generator.keras"
discriminator_path = f"models/{timestamp}/Discrimiator.keras"

# Save the model
generator.save(generator_path)
discriminator.save(discriminator_path)

In [ ]:
def guardar_imagenes_en_csv(imagenes_grises):
    """
    Guarda imágenes en escala de grises en un CSV con columnas: id, 0, ..., 4095.

    Args:
        imagenes_grises: array de forma (200, 64, 64) con imágenes en escala de grises.
        nombre_csv: nombre del archivo CSV a guardar.
    """
    num_imagenes = imagenes_grises.shape[0]
    pixeles_por_imagen = imagenes_grises.shape[1] * imagenes_grises.shape[2]

    # Aplanar cada imagen a un vector de 4096 elementos
    imagenes_aplanadas = imagenes_grises.reshape(num_imagenes, pixeles_por_imagen)

    # Crear DataFrame con columnas: id, 0, ..., 4095
    columnas = ['id'] + [str(i) for i in range(pixeles_por_imagen)]
    datos = np.column_stack((np.arange(1, num_imagenes + 1), imagenes_aplanadas))
    df = pd.DataFrame(datos, columns=columnas)

    # Guardar en CSV
    os.makedirs("submissions", exist_ok=True)
    timestamp = datetime.datetime.now().strftime("%m_%d_%H:%M")
    df.to_csv("submissions/GAN_"+timestamp+".csv", index=False)


guardar_imagenes_en_csv(generar_imagenes(generator, num_imagenes=200))